# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray


In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import re

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'config', 'local', 'met', 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
if "uk_food" in mongo.list_database_names():
    # Switch to the "uk_food" database
    db = mongo["uk_food"]
    # Retrieve the 'establishments' collection.
    collection = db["establishments"]
    print("Switched to 'uk_food' database successfully.")
else:
    print("'uk_food' database not found. Make sure it has been imported.")

Switched to 'uk_food' database successfully.


In [6]:
# review the collections in our new database
collections = db.list_collection_names()

if collections:
    print("Collections in the uk_food database:")
    for collection in collections:
        print("-", collection)
else:
    print("The uk_food database does not have any collections.")

Collections in the uk_food database:
- establishments


In [7]:
# review a document in the establishments collection
print(db.establishments.find_one())

{'_id': ObjectId('64b985d0526f7401b0d100a2'), 'FHRSID': 551803, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000070948', 'BusinessName': 'The Tea Room', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'The Pines Garden', 'AddressLine2': 'Beach Road', 'AddressLine3': 'St Margarets Bay', 'AddressLine4': 'Kent', 'PostCode': 'CT15 6DZ', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2021-08-17T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.383298', 'latitude': '51.148133'}, 'RightToReply': '', 'Distance': 4587.362402580997, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCou

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [10]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant_data)


In [11]:
# Check that the new restaurant was inserted
inserted_id = insert_result.inserted_id
print("Inserted document ID:", inserted_id)

# Retrieve the inserted restaurant data using the ID
inserted_restaurant = establishments.find_one({"_id": inserted_id})

# Print the retrieved data
print("Inserted restaurant data:")
print(inserted_restaurant)

Inserted document ID: 64b9864b597e3309ed5cbb7f
Inserted restaurant data:
{'_id': ObjectId('64b9864b597e3309ed5cbb7f'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {
    "BusinessType": "Restaurant/Cafe/Canteen"
}

# Define the projection to include only BusinessTypeID and BusinessType fields
projection = {
    "BusinessTypeID": 1,
    "BusinessType": 1,
    "_id": 0
}

# Find the document with the specified query and projection
result = establishments.find_one(query, projection)

# Print the result
print("BusinessTypeID and BusinessType for 'Restaurant/Cafe/Canteen':")
print(result)

BusinessTypeID and BusinessType for 'Restaurant/Cafe/Canteen':
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Define the query to find the BusinessTypeID for "Restaurant/Cafe/Canteen"
filter_query = {
    "BusinessName": "Penang Flavours"
}

# Define the update with the correct BusinessTypeID
update_query = {
    "$set": {
        "BusinessTypeID": "1"
    }
}

# Update the restaurant with the correct BusinessTypeID
update_result = establishments.update_one(filter_query, update_query)

# Check if the update was successful
if update_result.modified_count > 0:
    print("Restaurant updated successfully.")
else:
    print("Restaurant not found or no updates made.")


Restaurant updated successfully.


In [14]:
# Confirm that the new restaurant was updated
filter_query = {
    "BusinessName": "Penang Flavours"
}

# Retrieve the updated restaurant data
updated_restaurant = establishments.find_one(filter_query)

# Check if the update was successful and print the updated restaurant data
if updated_restaurant is not None:
    print("Updated restaurant data:")
    print(updated_restaurant)
else:
    print("Restaurant with name 'Penang Flavours' not found.")

Updated restaurant data:
{'_id': ObjectId('64b9864b597e3309ed5cbb7f'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '1', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments = db["establishments"]

# Define the query to find documents with "LocalAuthorityName" as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Count the documents that match the query
count_dover_restaurants = establishments.count_documents(query)

print("Number of documents with LocalAuthorityName as 'Dover':", count_dover_restaurants)

Number of documents with LocalAuthorityName as 'Dover': 994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
# Define the query to find documents with "LocalAuthorityName" as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Delete all documents that match the query
delete_result = establishments.delete_many(query)

# Check if the deletion was successful
if delete_result.deleted_count > 0:
    print(f"{delete_result.deleted_count} documents with LocalAuthorityName as 'Dover' were deleted.")
else:
    print("No documents with LocalAuthorityName as 'Dover' found to delete.")

994 documents with LocalAuthorityName as 'Dover' were deleted.


In [17]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}

# Find one document that matches the query
remaining_document = establishments.find_one(query)

if remaining_document:
    print("There are remaining documents with LocalAuthorityName as 'Dover'.")
else:
    print("No remaining documents with LocalAuthorityName as 'Dover'.")

No remaining documents with LocalAuthorityName as 'Dover'.


In [18]:
# Check that other documents remain with 'find_one'
query = {}

# Find one document that matches the query
remaining_document = establishments.find_one(query)

if remaining_document:
    print("There are remaining documents in the 'establishments' collection.")
else:
    print("No remaining documents found in the 'establishments' collection.")

There are remaining documents in the 'establishments' collection.


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
query = {
    "$or": [
        {"Latitude": {"$type": "string"}},
        {"Longitude": {"$type": "string"}}
    ]
}

# Update the documents to convert latitude and longitude to decimal numbers
update_query = {
    "$set": {
        "Latitude": {"$toDouble": "$Latitude"},
        "Longitude": {"$toDouble": "$Longitude"}
    }
}

update_result = establishments.update_many(query, update_query)

# Print the number of documents updated
print(f"{update_result.modified_count} documents updated.")

0 documents updated.


Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [21]:
# Change the data type from String to Integer for RatingValue
query = {"RatingValue": {"$type": "string"}}

# Update the documents to convert "RatingValue" to integer numbers
update_query = [
    {
        "$set": {
            "RatingValue": {"$toInt": "$RatingValue"}
        }
    }
]

update_result = establishments.update_many(query, update_query)

# Print the number of documents updated
print(f"{update_result.modified_count} documents updated.")

34694 documents updated.


In [22]:
# Check that the coordinates and rating value are now numbers
# Find one document to check the data types
sample_document = establishments.find_one()

if sample_document is not None:
    # Debug: Print the entire sample_document to understand its contents
    pprint(sample_document)

    # Access the nested "geocode" dictionary to get the Latitude and Longitude
    geocode = sample_document.get("geocode", {})
    latitude_type = type(geocode.get("latitude"))
    longitude_type = type(geocode.get("longitude"))

    # Check if "RatingValue" exists and handle None values
    rating_type = None
    if "RatingValue" in sample_document:
        rating_type = type(sample_document["RatingValue"])

    print("Data types after conversion:")
    print("Latitude:", latitude_type)
    print("Longitude:", longitude_type)

    if rating_type is not None:
        print("RatingValue:", rating_type)
    else:
        print("RatingValue: None (or not present in the document)")
else:
    print("No documents found in the 'establishments' collection.")

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64b985d0526f7401b0d10385'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': No